Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 8.23 ms, sys: 742 µs, total: 8.98 ms
Wall time: 11.4 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

<Client: 'inproc://10.1.1.163/8331/1' processes=1 threads=6, memory=6.78 GiB>

1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 758 ms, sys: 73.3 ms, total: 831 ms
Wall time: 2.16 s


[1.2414491999057942,
 1.6320624545507625,
 1.3990972793439727,
 1.9598952903400662,
 1.3255524432151915,
 1.9875256021669299,
 1.2887081073359608,
 1.8051623353091988,
 1.5843322999671634,
 1.2113043093729448,
 1.4640379860333244,
 1.8051745704895876,
 1.0843350057845436,
 1.7328699035123694,
 1.2743335595686607,
 1.5643247719091695,
 1.2082536746844927,
 1.7284537401665663,
 1.7771444937229424,
 1.9113162847970961,
 1.9287237877001244,
 1.8098748694190268,
 1.0276760946357744,
 1.3561317507801098,
 1.489431947447902,
 1.3541808606002004,
 1.0654093478514182,
 1.5958106205082228,
 1.473323012862149,
 1.5428207595725143,
 1.3450465660206385,
 1.6305125496860622,
 1.2268108790355061,
 1.9722476961807485,
 1.2113612845917423,
 1.806533848516924,
 1.965516666728079,
 1.9811971878714432,
 1.6407166301478529,
 1.2687406079344175,
 1.046458980222806,
 1.2629650543420508,
 1.3260034886521321,
 1.148009817048929,
 1.8270848973041924,
 1.8678530658171921,
 1.2126242372166511]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.4561612694527746,
 1.9823662720638402,
 1.3379236502761573,
 1.9065492867802285,
 1.6945596663293,
 1.5353757783466866,
 1.3947899163153994,
 1.4590522872504206,
 1.2767549329491705,
 1.9537054049086746,
 1.3462383285818558,
 1.6757709434290637,
 1.3951666215757768,
 1.7873053845028033,
 1.080858024908149,
 1.2239976897338538,
 1.0723516429680005,
 1.6194127741183169,
 1.871400267371448,
 1.2761834885096082,
 1.6468950818815697,
 1.4903723409361174,
 1.2773786598656287,
 1.1267459066326482,
 1.656604350975369,
 1.5211499528657697,
 1.36879563056781,
 1.9319024676089926,
 1.0147093173644683,
 1.7997008233145917,
 1.1379907251715515,
 1.144279813668272,
 1.850839695114874,
 1.2508699709049154,
 1.2191466650304297,
 1.319684124600161,
 1.4276202368046222,
 1.6154128594174795,
 1.6585757026176373,
 1.1329004227242345,
 1.2106502387832543,
 1.283364319510929,
 1.245676221246191,
 1.188514110665801,
 1.8118663623383586,
 1.7759963533629173,
 1.93956289586986,
 1.206851259615674,
 1.728950

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 321 ms, sys: 29.3 ms, total: 350 ms
Wall time: 1.95 s
